<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第三章: Attention

本章所需要的包

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
#导入并确认库

torch version: 2.6.0+cu126


- LLM的核心:Attention
- 译者:可以直接看论文呀!
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 长序列的建模

- 没有代码
- 逐字翻译文本通常不可行，因为源语言和目标语言在语法结构上存在差异：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- 在Transformer模型出现之前，机器翻译任务主要依赖于编码器(encoder)-解码器(decoder)架构的循环神经网络（RNNs）。
- 在这种架构中，编码器逐词处理源语言序列，并通过隐藏状态（神经网络中的中间层）生成输入序列的表示：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 注意力机制高效捕获数据关系

- 本节不涉及代码。
- 借助注意力机制，文本生成解码器能够选择性地关注所有输入token，从而在生成特定输出token时，动态分配不同输入token的重要性系数

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- Transformer中的自注意力机制是一种关键技术，它通过让序列中的每个位置与其他所有位置交互并计算相关性，从而增强输入表示的上下文信息。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 自注意力关注的不同部分

### 3.3.1 无可变参数的自注意力模型

- 本节介绍了一种高度简化的自注意力变体，不包含任何可训练的权重。
- 该变体仅用于说明目的，并非Transformer中实际使用的注意力机制。
- 下一节（3.3.2节）将扩展此简易模型，实现真正的自注意力机制。
- 假设给定一个输入序列 $x^{(1)}$ 到 $x^{(T)}$：
  - 输入是一个文本（例如，一句已被处理为token嵌入的句子，如“Your journey starts with one step”），具体处理方法在第2章中已有描述。
  - 例如，$x^{(1)}$ 是表示单词“Your”的d维向量，以此类推。

- **目标：** 为输入序列中的每个元素 $x^{(i)}$（从 $x^{(1)}$ 到 $x^{(T)}$）计算上下文向量 $z^{(i)}$（$z$ 和 $x$ 的维度相同）。
    - 上下文向量 $z^{(i)}$ 是对输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权求和。
    - 上下文向量是针对特定输入的“上下文”相关表示。
      - 以第二个输入 $x^{(2)}$ 为例，说明具体计算过程。
      - 第二个上下文向量 $z^{(2)}$ 是对所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权求和，权重由相对于 $x^{(2)}$ 的注意力权重决定。
      - 注意力权重决定了每个输入元素对 $z^{(2)}$ 的贡献程度。
      - 简而言之，$z^{(2)}$ 是 $x^{(2)}$ 的增强版本，融合了与当前任务相关的所有其他输入元素的信息。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- （请注意，此图中的数字已截断至小数点后一位，以减少视觉干扰；其他图表中的数值也可能经过类似处理。）

- 按照惯例，未归一化的注意力值称为 **“注意力得分”**，而归一化后总和为1的注意力得分称为 **“注意力权重”**。

- 下方代码逐步演示了上图的操作过程

<br>

- **步骤 1：** 计算未归一化的注意力得分 $\omega$
- 假设使用第二个输入token作为查询，即 $q^{(2)} = x^{(2)}$，通过点积计算未归一化的注意力得分：
    - $\omega_{21} = x^{(1)} \cdot q^{(2)\top}$
    - $\omega_{22} = x^{(2)} \cdot q^{(2)\top}$
    - $\omega_{23} = x^{(3)} \cdot q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} \cdot q^{(2)\top}$
- 其中，$\omega$ 是希腊字母“欧米伽”，表示未归一化的注意力得分。
    - 在 $\omega_{21}$ 中，下标“21”表示以第2个元素为查询，与第1个元素计算得分。

- 假设我们有以下输入句子，该句子已根据第3章的描述嵌入到3维向量中（此处我们使用了一个非常小的嵌入维度进行说明，以便内容可以显示在页面上）： 

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
#对于一句话中的每个单词定义了一个三维的向量

- （在本书中，我们遵循机器学习和深度学习的常见惯例：训练样本以行表示，特征值以列表示；对于上述张量，每一行表示一个词，每一列表示一个嵌入维度。）

- 本节的主要目标是演示如何以第二个输入序列 $x^{(2)}$ 作为查询，计算其上下文向量 $z^{(2)}$。

- 图中展示了该过程的第一步，即通过点积操作计算 $x^{(2)}$ 与所有其他输入元素之间的注意力得分 $\omega$。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 我们以输入序列中的第2个元素 $x^{(2)}$ 为例，计算其上下文向量 $z^{(2)}$；稍后会将此方法推广至计算所有上下文向量。
- 第一步是通过计算查询 $x^{(2)}$ 与所有输入token的点积，得到未归一化的注意力得分：

In [3]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
#建立一个未初始化的张量来记录注意力得分
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) 
    # 相似性度量计算attention分数
    # 从公式上看也就是点乘

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 补充说明：点积本质上是逐元素相乘并将所得积相加的一种简写表示：

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]
    #累加Key*Query的成绩

print(res)
print(torch.dot(inputs[0], query))
#相当于解释了一遍点成的内部原理

tensor(0.9544)
tensor(0.9544)


- **步骤 2：** 将未归一化的注意力得分（“欧米伽”，$\omega$）归一化，使其总和为1。
- 以下是一种简单的方法，用于将未归一化的注意力得分归一化：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum() 
#归一化,这里是属于加权式的归一化

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实践中，使用softmax函数进行归一化更为常见，因为它能够更好地处理极端值，并且在训练过程中具有更理想的梯度特性，因此推荐使用。
- 下面是一个简单的softmax函数实现，用于缩放并对向量元素进行归一化，使它们的和为1：

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())
##用SoftMax做归一化, 处理好极端值
#有合理的梯度数据表现力

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上述简单实现可能会因输入值过大或过小而遭遇数值不稳定问题，导致溢出或下溢。
- 因此，在实践中，建议使用PyTorch内置的softmax函数，因为它经过高度优化，性能更佳：

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())
#用torch优化过的softmax对边缘值也挺友好的

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤 3**：通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘，并对结果向量求和，计算上下文向量 $z^{(2)}$：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [8]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
#创造一个内容的零向量
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
    #把不同内容的向量+起来

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有token的attention score

#### 将其推广到所有输入序列标记：

- 上面，我们为输入2计算了注意力权重和上下文向量（如下图中高亮的行所示）。
- 接下来，我们将此计算推广到所有输入序列标记，计算对应的注意力权重和上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- （请注意，图中的数字已四舍五入到小数点后两位；每一行的数值应相加为1.0或100%；其他图中的数字也进行了类似处理。）

- 在自注意力机制中，首先计算注意力得分，随后对这些得分进行归一化，得到总和为1的注意力权重。
- 接着，利用这些注意力权重对输入进行加权求和，生成上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- 对所有成对元素应用之前的**步骤 1**，计算未归一化的注意力得分矩阵：

In [9]:
attn_scores = torch.empty(6, 6)
#建立个空表来储存相关联程度

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
        #一点点计算相关性并输入表格
print(attn_scores)
#事实上就是实现了两个单词之间的关联度列表输出

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 如果是矩阵相乘那么更有效率

In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)
#有简单的方法整合方法计算

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与**第二步**相似, 我们对每一行都要归一化操作:

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)
#归一化处理

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 一个快速验证

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))
#验证一下大家加起来都是1

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 用**step 3** 计算所有的向量:

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)
#重复了上一个操作

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


-  作为合理性检查，之前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上图的第二行找到：

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 可调整参数的自注意力机制

- 以下的概念框架展示了本节中开发的自注意力机制,以及这种机制是如何融入本书和本章的整体叙述与结构。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 手把手的计算attention的值

- 在本节中，我们实现了原始 Transformer 架构、GPT 模型以及大多数流行 LLM 中使用的自注意力机制。  
- 这种自注意力机制被称为“缩放点积注意力”（scaled dot-product attention）。  
- 整体思路与之前相似：  
  - 我们希望计算针对特定输入元素的上下文向量，即输入向量的加权和。  
  - 为此，我们需要生成注意力权重。  
- 如你所见，与之前介绍的基本注意力机制相比，只有一些细微差异：  
  - 最显著的区别是引入了在模型训练过程中更新的权重矩阵。  
  - 这些可训练的权重矩阵至关重要，它们使模型（尤其是注意力模块）能够学习生成“优质”的上下文向量。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 按照步骤实现自注意力机制，我们将首先介绍三个训练权重矩阵 $W_q$、$W_k$ 和 $W_v$。  
- 这三个矩阵用于通过矩阵乘法将嵌入的输入标记 $x^{(i)}$ 映射到查询向量、键向量和值向量：
- (译者: 分别是Query、Key、Value,专有名词)   

  - 查询向量：$q^{(i)} = W_q \,x^{(i)}$  
  - 键向量：$k^{(i)} = W_k \,x^{(i)}$  
  - 值向量：$v^{(i)} = W_v \,x^{(i)}$  


- 输入 $x$ 和查询向量 $q$ 的嵌入维度可以相同，也可以不同，具体取决于模型的设计和实现方式。
- 在 GPT 模型中，输入和输出维度通常是相同的，但为了便于示范并更好地理解计算过程，这里我们选择了不同的输入和输出维度：

In [15]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- 下面，我们初始化三个权重矩阵；请注意，为了简化输出并便于示范，我们将 `requires_grad=False`
- 但如果我们要在模型训练中使用这些权重矩阵，应将 `requires_grad=True`，以便在训练过程中更新这些矩阵。

In [16]:
torch.manual_seed(123)
#固定随机种子确保可复现性

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
#初始化三个矩阵来存放
#不要求梯度降低了复杂度

- 计算这三个向量值

In [17]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value
#点积计算
print(query_2)

tensor([0.4306, 1.4551])


- 我们可以清晰地看到,embedding被降维了:

In [18]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)
#中途检验下

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步 **步骤 2** 中，我们通过计算查询向量和每个键向量之间的点积来计算未归一化的注意力得分：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [19]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 因为我们有六个输入,所以我们有六个attention score

In [20]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)
#计算注意力跟query值

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 接下来，在 **步骤 3** 中，我们使用之前提到的 softmax 函数计算注意力权重（归一化后的注意力得分，总和为 1）。
- 与之前的不同之处在于，我们现在通过将注意力得分除以嵌入维度的平方根 $\sqrt{d_k}$（即 `d_k**0.5`）来对注意力得分进行缩放：

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
#压缩函数, 有利于储存与比较
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- 在**第四步**, 我们可以计算每一个token的向量了:

In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 自注意模块

- 下面是代码

In [23]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        #定义QKV的随机矩阵

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        #模型的训练传递
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- 我们可以使用 PyTorch 的 `Linear` 层简化上述实现，禁用偏置项后，`Linear` 层相当于矩阵乘法。
- 使用 `nn.Linear` 替代手动使用 `nn.Parameter(torch.rand(...))` 的一个主要优势是，`nn.Linear` 具有推荐的权重初始化方案，这有助于模型训练更加稳定。

In [24]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        #权重初始化

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T 
        #Query跟Key的计算 得出初始的分数传递到后面进行归一化操作
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        #直接基于注意力对于文本计算
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- `SelfAttention_v1` 和 `SelfAttention_v2` 会给出不同的输出，因为它们使用了不同的初始权重矩阵。

## 3.5 对未出现的信息的隐藏

- 在casual attention，对角线以上的注意力权重被掩蔽，确保在计算上下文向量时，LLM 无法利用位置的信息来调整注意力权重。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 因果自注意力机制

- 在这一节中，我们将把之前的自注意力机制转换为因果自注意力机制。
- 因果自注意力确保模型在预测序列中某个位置的值时，仅依赖于前面已知位置的输出，而不依赖于后续位置。
- 换句话说，这确保了每个下一个词的预测仅依赖于前面的词。
- 为了实现这一点，对于每个给定的标记，我们会将“未知的信息”（即输入文本中当前token之后的token）掩蔽掉：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 为了说明和实现因果自注意力，让我们使用上一节中的注意力得分和权重：

In [25]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)
#用mask的数据重新算了一次

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 隐藏未知信息的attention score最简单的方法是通过 PyTorch 的 `tril` 函数进行掩蔽，其中主对角线以下的元素（包括对角线本身）设置为 1，主对角线以上的元素设置为 0：

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
#Mask矩阵,直接保留Diagonal下部分的,上部分掩盖掉
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与这个mask相乘，以将对角线以上的注意力得分置为零：

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)
#简单的效果图

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 如果在 softmax 之后进行掩蔽，它会破坏 softmax 所创建的概率分布。
- softmax 确保所有输出值的总和为 1。
- 如果在 softmax 之后进行掩蔽，就需要重新归一化输出，确保其总和为 1，这会使过程更加复杂，并可能带来意想不到的效果。

- 我们可以用以下方式确保所有的数据都是归一化的

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)
#掩码之后的softmax

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 尽管我们在技术上已经完成了因果注意力机制的编码，但让我们简要地探讨一种更高效的方法，以实现与上述相同的效果。
- 因此，在注意力得分进入 softmax 函数之前，我们可以将对角线以上的未归一化注意力得分用负无穷大进行掩蔽，而不是将其置零并重新归一化：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
#创建一个全1的三角,去上部分变成0
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
#有掩码的地方变为负无穷
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 结果显然是归一化的

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 使用Dropout防止过拟合

- 此外，我们还应用了丢弃（Dropout）来减少训练过程中的过拟合。
- dropout可以应用于多个位置：
  - 例如，在计算注意力权重之后；
  - 或在将注意力权重与值向量相乘之后。
- 在这里，我们选择在计算注意力权重之后应用丢弃掩码，因为这种做法更为常见。

- 另外，在此示例中，我们使用了50%的丢弃率，这意味着随机屏蔽掉一半的注意力权重。（在后续训练GPT模型时，我们会使用更低的丢弃率，例如0.1或0.2。）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 如果我们应用0.5（50%）的丢弃率，未被抛弃的值将相应地被缩放一个因子，1/0.5 = 2。
- 这种缩放通过公式 1 / (1 - `dropout_rate`) 计算得出。

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) 
# dropout rate of 50%丢包率doge
example = torch.ones(6, 6) 
# create a matrix of ones满的6*6矩阵被1包圆了

print(dropout(example))
#输出需要被放大相应的倍数,为了维持恒定

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [32]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


- 生成的输出可能会因操作系统的不同而有所不同；
- 你可以在 [PyTorch 问题追踪器](https://github.com/pytorch/pytorch/issues/121595) 上了解更多内容。

### 3.5.3 实现一个简洁的因果自注意力

- 现在，我们准备实现一个完整的自注意力机制，包含因果掩码和dropout。
- 另一项任务是实现代码以处理包含多个输入的批次，确保我们的 `CausalAttention` 类能够支持第二章中实现的数据加载器所生成的批量输出。
- 为了简化起见，我们通过复制输入文本示例来模拟批量输入：

In [33]:
batch = torch.stack((inputs, inputs), dim=0)
#相同的tensor按照指定维度堆叠
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


- 缩放因子  \sqrt{d}  的引入解决了注意力机制中的数值不稳定问题。
- 它确保了即使嵌入维度  d  较大，点积得分也能被合理地控制在一个适当范围，方便 Softmax 生成平滑的注意力分布，且梯度不会过大或过小。

In [34]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        #初始化定义网络结构和参数
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New
        #定义QKV并对进行dropout防止过拟合
        #注册mask向量,对未来进行负无穷的拟合
    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        #提取batch的大小、token的数量、跟宽度
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        #进行运算计算
        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        #通过点积来计算attention的数值
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1## 缩放因子 √d，用于稳定梯度
        )
        #在时间顺序上进行mask确保信息不会被泄露
        attn_weights = self.dropout(attn_weights) # New
        #防止过拟合的dropout处理方式
        context_vec = attn_weights @ values
        # 根据注意力权重计算上下文向量
        return context_vec

torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Dropout仅在训练时要使用,验证时不需要

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 拓展单头至多方注意

### 3.6.1 堆叠多个单头注意力层

- 以下是之前实现的自注意力机制总结（为简化起见，未展示因果和dropout掩码）：

- 这种机制也称为单头注意力：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 我们通过堆叠多个单头注意力模块来构建多头注意力模块：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 多头注意力的核心思想是使用不同的学习到的线性投影，并行地多次运行注意力机制。这使得模型能够在不同位置同时关注来自不同表示子空间的信息。

在 Python 中，super().__init__() 是一种调用父类（基类）构造函数的方法，常用于类继承的场景中。它确保子类能够正确初始化父类的属性和方法。

In [35]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__() 
        #多个实例,每个都是一个头
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    #模型的训练


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上述实现中，嵌入维度为4，因为我们将 `d_out=2` 作为键、查询和值向量以及上下文向量的嵌入维度。由于使用了2个注意力头，输出嵌入维度为 2 * 2 = 4。

### 3.6.2 利用权重拆分实现多头注意力

- 尽管上述实现是一种直观且功能完整的多头注意力机制（通过封装之前的单头注意力 `CausalAttention` 实现），我们仍可以编写一个独立的 `MultiHeadAttention` 类来实现相同的功能。

- 在这个独立的 `MultiHeadAttention` 类中，我们不会将单个注意力头进行拼接。
- 相反，我们会创建独立的 W_query、W_key 和 W_value 权重矩阵，并将它们拆分为每个注意力头的单独矩阵：

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"
        #确保是可以被整除的
            

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim
        #初始化头的维度、数量
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        #头的输出结合线性层
        self.dropout = nn.Dropout(dropout)
        #进行dropout防止过拟合
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
        # 上三角掩码，确保因果性

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)
        #把输出的维度拆成头*头大小
        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        #转制维度,听说是为了更好的计算注意力
        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        # 计算缩放点积注意力
        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head
        # 将掩码缩减到当前 token 数量，并转换为布尔型
        # 进而实现动态遮蔽,所以不用另开好几个数组
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # 遮蔽矩阵
        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        #归一化
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        #头的合并
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        #对上下文向量的形状进行调整，确保输出的形状
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，上述实现本质上是 `MultiHeadAttentionWrapper` 的重写版本，并且更加高效。  
- 生成的输出看起来略有不同，因为随机权重初始化有所不同，但两者都是完全可用的实现，可以在我们将在后续章节中实现的 GPT 类中使用。  
- 另外，值得注意的是，我们在上面的 `MultiHeadAttention` 类中添加了一个线性投影层（`self.out_proj`）。这只是一个线性变换，不改变维度。在 LLM 实现中，使用这样的投影层是标准做法，但它并非严格必要（近期的研究表明，去除该层不会影响模型的表现；请参阅本章末尾的进一步阅读部分）。  


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 请注意，如果你对上述内容的紧凑和高效实现感兴趣，可以考虑使用PyTorch中的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类。

- 由于上述实现初看起来可能有些复杂，我们来看一下执行 `attn_scores = queries @ keys.transpose(2, 3)` 时会发生什么：

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))
#每个注意力头中，输出矩阵的值表示每个 token 对其他 token 的相关性。
#模型可以计算 token 之间的相关性，为注意力分布的生成奠定基础

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现将处理四维输入张量，使得矩阵乘法在最后两个维度（`num_tokens`, `head_dim`）之间进行，然后对各个头进行重复计算。  

- 例如，以下方法提供了一种更紧凑的方式来分别计算每个头的矩阵乘法：  


In [38]:
first_head = a[0, 0, :, :]
#定义第一个头
first_res = first_head @ first_head.T
#第一个矩阵的自相关性
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 总结与收获

- 请参阅 [./multihead-attention.ipynb](./multihead-attention.ipynb) 代码笔记本，它是数据加载器（第2章）的简洁版本，加上我们在本章实现的多头注意力类，后续章节中训练GPT模型时将需要使用。
- 你可以在 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 中找到习题解答。